In [2]:
from Financial_data_crawler.db import clients

conn = clients.get_mongodb_news_conn()

In [3]:
db_p= conn['Comp'].Profile

In [3]:
from bs4 import BeautifulSoup

res = '''<td class="t4t1" id="oAddCheckbox" nowrap="">
<a href="javascript:Link2Stk('00878');">00878國泰永續高股息</a>
</td>
'''

soup = BeautifulSoup(res, 'html.parser')
soup.find('a').text


'00878國泰永續高股息'

In [4]:
test_str = """
    <SCRIPT LANGUAGE=javascript>
    <!--
    GenLink2stk('AS1795','美時');
    //-->
    </SCRIPT>
    """
soup_obj = BeautifulSoup(test_str,'html.parser')
soup_obj.find('script')

<script language="javascript">
    <!--
    GenLink2stk('AS1795','美時');
    //-->
    </script>

In [12]:
import re
pattern = '^[A-Za-z0-9]+'
res = re.search(pattern,'00878國泰永續高股息')
res.group(0)
res.span()[1]

5

In [7]:
res

<re.Match object; span=(0, 5), match='00878'>

In [5]:
import psycopg2
import psycopg2.extras

In [2]:
conn = psycopg2.connect(dbname="postgres", user="postgres", password="postgres",host="localhost",port="5432")

In [6]:
dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
dict_cur.execute("SELECT * FROM Raw_Material")


In [7]:
rec = dict_cur.fetchall()

In [13]:
import datetime
from Financial_data_crawler.db.clients import MongoClient
from Financial_data_crawler.db.RMModel import Raw_Material

client=MongoClient('Scrapy','Raw_Material')
for i, n in enumerate(rec):
    d={}
    date=datetime.datetime.strftime(n[1], "%Y/%m/%d")
    d['Date']=date
    d['Material_Name']=n[2]
    d['Price']=n[3]
    Raw_Material.objects(Date=date, Material_Name=n[2]).modify(upsert=True, **d)

In [5]:
from Financial_data_crawler.db.clients import MongoClient
from Financial_data_crawler.db.ChipModels import Broker_Info

client=MongoClient('Scrapy','sel_broker')

In [10]:
brokers = Broker_Info.objects()


In [13]:

brokers = Broker_Info.objects()

base_url = r'/z/zg/zgb/zgb0.djhtm?'
date = None
if date:
    date_str = f'&e={date}&f={date}' # Only parse daily info
else:
    date_str = ''

start_urls = []

for broker in brokers:
    broker_code = broker['BrokerCode']
    broker_name = broker['BrokerName']

    if broker['BrokerBranch'] is None:
        broker_d = {broker_code: broker_name}

    else:
        broker_d = broker['BrokerBranch']

    for branch in broker_d.keys():
        if re.search('[A-Za-z]', branch):
            branch_code = ''.join([format(ord(c), '02x').zfill(4) for c in branch])
        else:
            branch_code = branch

        brach_name = broker['BrokerBranch'][branch]
        url_vol =  base_url + f'a={broker_code}&b={branch_code}&c=E' + date_str
        url_amt = base_url + f'a={broker_code}&b={branch_code}&c=B' + date_str
        meta = {'BrokerCode': broker_code,
                'BrokerName': broker_name,
                'BranchName': brach_name,
                'BranchCode': branch}

        start_urls.append({'url':url_vol,'meta':meta})
        start_urls.append({'url':url_amt,'meta':meta})


len(start_urls)


1714

In [8]:
import re

base_url = r'z/zg/zgb/zgb0.djhtm?'
start_urls = []

for broker in brokers:
    broker_code = broker['BrokerCode']
    broker_name = broker['BrokerName']

    if broker['BrokerBranch'] is None:
        broker_d = {broker_code:broker_name}
    
    else: 
        broker_d =broker['BrokerBranch']

    for branch in broker['BrokerBranch'].keys():
        if re.search('[A-Za-z]',branch):
            branch_code = ''.join([format(ord(c), '02x').zfill(4) for c in branch])
        else:
            branch_code = branch
            
        brach_name = broker['BrokerBranch'][branch]
        start_urls.append({'url':base_url+f'a={broker_code}&b={branch_code}',
                            'meta':{'BrokerCode':broker_code,
                                    'BrokerName':broker_name,
                                    'BranchName':brach_name,
                                    'BranchCode':branch}})


In [9]:
start_urls

[{'url': 'z/zg/zgb/zgb0.djhtm?a=6010&b=6010',
  'meta': {'BrokerCode': '6010',
   'BrokerName': '(牛牛牛)亞證券',
   'BranchName': '(牛牛牛)亞證券',
   'BranchCode': '6010'}},
 {'url': 'z/zg/zgb/zgb0.djhtm?a=6620&b=6620',
  'meta': {'BrokerCode': '6620',
   'BrokerName': '口袋證券',
   'BranchName': '口袋證券',
   'BranchCode': '6620'}},
 {'url': 'z/zg/zgb/zgb0.djhtm?a=1030&b=1030',
  'meta': {'BrokerCode': '1030',
   'BrokerName': '土銀',
   'BranchName': '土銀',
   'BranchCode': '1030'}},
 {'url': 'z/zg/zgb/zgb0.djhtm?a=1030&b=1031',
  'meta': {'BrokerCode': '1030',
   'BrokerName': '土銀',
   'BranchName': '土銀-台中',
   'BranchCode': '1031'}},
 {'url': 'z/zg/zgb/zgb0.djhtm?a=1030&b=1032',
  'meta': {'BrokerCode': '1030',
   'BrokerName': '土銀',
   'BranchName': '土銀-台南',
   'BranchCode': '1032'}},
 {'url': 'z/zg/zgb/zgb0.djhtm?a=1030&b=1033',
  'meta': {'BrokerCode': '1030',
   'BrokerName': '土銀',
   'BranchName': '土銀-高雄',
   'BranchCode': '1033'}},
 {'url': 'z/zg/zgb/zgb0.djhtm?a=1030&b=1034',
  'meta': {'Broke

In [6]:
import pandas as pd
broker_branch = pd.read_csv(r'C:\Users\s3309\Downloads\證券商資訊表.csv',encoding='utf-8',on_bad_lines='skip')

In [ ]:
res = ""

In [26]:
for i in range(len(broker_branch)):
    broker_name = broker_branch['券商名稱'][i].replace(r' ','').replace('臺銀證券','臺銀')\
        .replace('台灣匯立','台灣匯立證券').replace('犇','(牛牛牛)').replace('口袋','口袋證券').replace('聯邦','聯邦商銀')
    broker_code = broker_branch['券商代碼'][i].replace(r' ','')

    try:
        broker = Broker_Info.objects(BrokerName=broker_name.split('-')[0]).first()
        del broker.BrokerBranch['BranchCode']
        del broker.BrokerBranch['BranchName']
        broker.save()
    except:
        print('Error----',broker_name)
        continue
    


Error---- 合庫-台中
Error---- 合庫-台南
Error---- 合庫-高雄
Error---- 合庫-嘉義
Error---- 合庫-基隆
Error---- 合庫-彰化
Error---- 合庫-鳳山
Error---- 合庫-新竹
Error---- 合庫-自強
Error---- 合庫-桃園
Error---- 合庫-西台中
Error---- 合庫-三重
Error---- 合庫-國際證券
Error---- 土銀-台中
Error---- 土銀-台南
Error---- 土銀-高雄
Error---- 土銀-嘉義
Error---- 土銀-新竹
Error---- 土銀-玉里
Error---- 土銀-花蓮
Error---- 土銀-和平
Error---- 土銀-士林
Error---- 土銀-建國
Error---- 土銀-彰化
Error---- 土銀-白河
Error---- 土銀-南港
Error---- 臺銀-鳳山
Error---- 臺銀-臺南
Error---- 臺銀-民權
Error---- 臺銀-新竹
Error---- 臺銀-臺中
Error---- 臺銀-高雄
Error---- 臺銀-金山
Error---- 台灣企銀-台中
Error---- 台灣企銀-台南
Error---- 台灣企銀-九如
Error---- 台灣企銀-嘉義
Error---- 台灣企銀-太平
Error---- 台灣企銀-屏東
Error---- 台灣企銀-竹北
Error---- 台灣企銀-豐原
Error---- 台灣企銀-岡山
Error---- 台灣企銀-民雄
Error---- 台灣企銀-建成
Error---- 台灣企銀-三民
Error---- 臺灣企銀-桃園
Error---- 台灣企銀-北高雄
Error---- 臺灣企銀-埔墘
Error---- 日盛-忠孝
Error---- 日盛-台南
Error---- 日盛-台中
Error---- 日盛-內湖
Error---- 日盛-板橋
Error---- 日盛-雙和
Error---- 日盛-嘉義
Error---- 日盛-高雄
Error---- 日盛-士林
Error---- 日盛-木柵
Error---- 日盛-中壢
Error---- 日盛-園區
Error-

In [4]:
for key in broker.keys():
    Broker_Info.objects(BrokerName=key).modify(upsert=True, BrokerName=key, BrokerCode=broker[key])

In [5]:
z/zg/zgb/zgb0.djhtm?a=6160&b=6160
z/zg/zgb/zgb0.djhtm?a=6160&b=6161
z/zg/zgb/zgb0.djhtm?a=6160&b=6162
z/zg/zgb/zgb0.djhtm?a=6160&b=6163
z/zg/zgb/zgb0.djhtm?a=6160&b=6164
z/zg/zgb/zgb0.djhtm?a=6160&b=6165

a.com.tw/z/zg/zgb/zgb0.djhtm?a=6160&b=6160
a.com.tw/z/zg/zgb/zgb0.djhtm?a=6160&b=6161
a.com.tw/z/zg/zgb/zgb0.djhtm?a=6160&b=6162
b.com.tw/z/zg/zgb/zgb0.djhtm?a=6160&b=6163
b.com.tw/z/zg/zgb/zgb0.djhtm?a=6160&b=6164
b.com.tw/z/zg/zgb/zgb0.djhtm?a=6160&b=6165


In [2]:
'779J'.isalpha()

True

In [36]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(res,'html.parser')

In [82]:
date = soup.find('div', {'class': 't11'})
date.text[-8:]

'20230210'

In [76]:
table = soup.find_all('table')[4]

In [77]:
header_line = table.find_all('tr')[1]
header = [h.text for h in header_line.find_all('td')]
header

['券商名稱', '買進金額', '賣出金額', '差額']

In [35]:
name_str = table.find_all('tr')[2].find('script').text

s = "\n<!--\n\tGenLink2stk('AS0050','元大台灣50');\n//-->\n"
pattern = r"GenLink2stk\('(\w+)','([^']+)'\);"
match = re.search(pattern, s)
if match:
    result = [match.group(1), match.group(2)]
    print(result)

In [51]:
[t.text for t in table.find_all('tr')[2].find_all('td')[1:]]



['149,887', '16,542', '133,345']

In [50]:
table.find_all('tr')[2].find_all('td')

[<td class="t4t1" id="oAddCheckbox" nowrap="">
 <script language="javascript">
 <!--
 	GenLink2stk('AS0050','元大台灣50');
 //-->
 </script>
 </td>,
 <td class="t3n1" nowrap="">149,887</td>,
 <td class="t3n1" nowrap="">16,542</td>,
 <td class="t3n1" nowrap="">133,345</td>]

In [44]:
x is None

True

['AS0050', '元大台灣50']


In [46]:
match